## Testing Async Feature vs Sync
This notebook demonstrates how to deploy and test a Nuclio function with async feature.
It configures the Nuclio function to handle incoming events, and allocates workers with multiple connections to handle the requests in async mode. 

In [1]:
import mlrun
import time
import inspect

In [2]:
project = mlrun.get_or_create_project("async-nuclio-test3",user_project=True,allow_cross_project=True)

> 2025-11-09 10:51:57,715 [info] Project loaded successfully: {"project_name":"async-nuclio-test3-guyl"}


In [3]:
%%writefile async_nuclio3.py
import asyncio
import json
from datetime import datetime

async def handler(context, event):
    body = event.body

    # handle all three cases: dict, bytes, str
    if isinstance(body, dict):
        data = body
    else:
        if isinstance(body, bytes):
            body = body.decode("utf-8")
        try:
            data = json.loads(body)
        except Exception:
            data = {"raw": body}

    worker = context.worker_id
    context.logger.info(f"Event: {data}, {event.path} with headers: {event.headers}")

    await asyncio.sleep(1)

    resp = {
        "test": data.get("test"),
        "send_at": data.get("send_at"),
        "worker_id": worker,
        "end_time": datetime.now().isoformat(),
    }

    return context.Response(
        body=json.dumps(resp),
        headers={"Content-Type": "application/json"},
        status_code=200,
    )


Overwriting async_nuclio3.py


## Test Nuclio Async Mode
 Testing with the following parameters: `events 20`, `maxConnectionsNumber 20`, `1 worker`, `connectionAvailabilityTimeout 60s`. Testing a number of events distributed to 1 worker, and see the responses time for each event, over all time took to finish handling events and the success and failed hadlings of events. The events should be recieved in the same time and handled at the same time.

In [4]:
async_func3 = project.set_function(func="async_nuclio3.py",name="async-nuclio3",image="mlrun/mlrun",kind="nuclio")

In [5]:
async_func3.with_http(workers=1, worker_timeout=30)
async_func3.spec.min_replicas = 1
async_func3.spec.max_replicas = 1
async_func3.spec.config["spec.triggers.http"]["mode"] = "async"
async_func3.spec.config["spec.triggers.http"]["async"]={"maxConnectionsNumber": 20,"minConnectionsNumber":20, "connectionAvailabilityTimeout":"60s"}
async_func3.set_config("spec.eventTimeout", "50s") 

> 2025-11-09 10:51:57,739 [warning] Adding HTTP trigger despite the default HTTP trigger creation being disabled


In [6]:
async_func3.deploy()

> 2025-11-09 10:51:57,751 [info] Starting remote function deploy
2025-11-09 10:51:58  (info) Deploying function
2025-11-09 10:51:58  (info) Building
2025-11-09 10:51:58  (info) Staging files and preparing base images
2025-11-09 10:51:58  (warn) Using user provided base image, runtime interpreter version is provided by the base image
2025-11-09 10:51:58  (info) Building processor image
2025-11-09 10:53:13  (info) Build complete
2025-11-09 10:53:23  (info) Function deploy complete
> 2025-11-09 10:53:28,828 [info] Successfully deployed function: {"external_invocation_urls":["async-nuclio-test3-guyl-async-nuclio3.default-tenant.app.cust-cs-il.iguazio-cd0.com/"],"internal_invocation_urls":["nuclio-async-nuclio-test3-guyl-async-nuclio3.default-tenant.svc.cluster.local:8080"]}


'http://async-nuclio-test3-guyl-async-nuclio3.default-tenant.app.cust-cs-il.iguazio-cd0.com/'

In [7]:
import asyncio
import aiohttp
from datetime import datetime
import pandas as pd

url = "http://async-nuclio-test3-guyl-async-nuclio3.default-tenant.app.cust-cs-il.iguazio-cd0.com/"

async def fetch(session, url, i):
    payload = {"test": i, "send_at": datetime.now().isoformat()}
    try:
        async with session.post(url, json=payload) as resp:
            text = await resp.text()
            return {
                "ok": resp.status == 200,
                "status": resp.status,
                "text": text,
                "i": i,
            }
    except Exception as e:
        return {
            "ok": False,
            "status": None,
            "text": str(e),
            "i": i,
        }

async def fetch(session, url, i):
    payload = {"test": i, "send_at": datetime.now().isoformat()}
    try:
        async with session.post(url, json=payload) as resp:
            text = await resp.text()
            return {
                "ok": resp.status == 200,
                "status": resp.status,
                "text": text,
                "i": i,
            }
    except Exception as e:
        return {
            "ok": False,
            "status": None,
            "text": str(e),
            "i": i,
        }

async def main():
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url, i) for i in range(20)]
        responses = await asyncio.gather(*tasks, return_exceptions=True)

    # do not count here, just return them all
    return responses

responses = await main()


In [13]:
print(responses)

[{'ok': True, 'status': 200, 'text': '{"test": 0, "send_at": "2025-11-09T10:53:28.889495", "worker_id": "0", "end_time": "2025-11-09T10:53:29.929812"}', 'i': 0}, {'ok': True, 'status': 200, 'text': '{"test": 1, "send_at": "2025-11-09T10:53:28.889842", "worker_id": "0", "end_time": "2025-11-09T10:53:29.927336"}', 'i': 1}, {'ok': True, 'status': 200, 'text': '{"test": 2, "send_at": "2025-11-09T10:53:28.889945", "worker_id": "0", "end_time": "2025-11-09T10:53:29.929249"}', 'i': 2}, {'ok': True, 'status': 200, 'text': '{"test": 3, "send_at": "2025-11-09T10:53:28.890166", "worker_id": "0", "end_time": "2025-11-09T10:53:29.929435"}', 'i': 3}, {'ok': True, 'status': 200, 'text': '{"test": 4, "send_at": "2025-11-09T10:53:28.890259", "worker_id": "0", "end_time": "2025-11-09T10:53:29.929634"}', 'i': 4}, {'ok': True, 'status': 200, 'text': '{"test": 5, "send_at": "2025-11-09T10:53:28.890339", "worker_id": "0", "end_time": "2025-11-09T10:53:29.929157"}', 'i': 5}, {'ok': True, 'status': 200, 'text

In [9]:
df = pd.DataFrame(responses)

In [14]:
display(df.head(20))  # first 20 rows
display(df.tail(10))  # last 10 rows


,ok,status,text,i,send_at,end_time,send_at_0_1s,end_time_0_1s
0,True,200,"{""test"": 0, ""send_at"": ""2025-11-09T10:53:28.88...",0,2025-11-09 10:53:28.889495,2025-11-09 10:53:29.929812,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
1,True,200,"{""test"": 1, ""send_at"": ""2025-11-09T10:53:28.88...",1,2025-11-09 10:53:28.889842,2025-11-09 10:53:29.927336,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
2,True,200,"{""test"": 2, ""send_at"": ""2025-11-09T10:53:28.88...",2,2025-11-09 10:53:28.889945,2025-11-09 10:53:29.929249,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
3,True,200,"{""test"": 3, ""send_at"": ""2025-11-09T10:53:28.89...",3,2025-11-09 10:53:28.890166,2025-11-09 10:53:29.929435,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
4,True,200,"{""test"": 4, ""send_at"": ""2025-11-09T10:53:28.89...",4,2025-11-09 10:53:28.890259,2025-11-09 10:53:29.929634,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
5,True,200,"{""test"": 5, ""send_at"": ""2025-11-09T10:53:28.89...",5,2025-11-09 10:53:28.890339,2025-11-09 10:53:29.929157,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
6,True,200,"{""test"": 6, ""send_at"": ""2025-11-09T10:53:28.89...",6,2025-11-09 10:53:28.890409,2025-11-09 10:53:29.927465,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
7,True,200,"{""test"": 7, ""send_at"": ""2025-11-09T10:53:28.89...",7,2025-11-09 10:53:28.890473,2025-11-09 10:53:29.929523,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
8,True,200,"{""test"": 8, ""send_at"": ""2025-11-09T10:53:28.89...",8,2025-11-09 10:53:28.890540,2025-11-09 10:53:29.926489,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
9,True,200,"{""test"": 9, ""send_at"": ""2025-11-09T10:53:28.89...",9,2025-11-09 10:53:28.890603,2025-11-09 10:53:29.927565,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900


,ok,status,text,i,send_at,end_time,send_at_0_1s,end_time_0_1s
10,True,200,"{""test"": 10, ""send_at"": ""2025-11-09T10:53:28.8...",10,2025-11-09 10:53:28.890668,2025-11-09 10:53:29.927110,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
11,True,200,"{""test"": 11, ""send_at"": ""2025-11-09T10:53:28.8...",11,2025-11-09 10:53:28.890727,2025-11-09 10:53:29.929035,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
12,True,200,"{""test"": 12, ""send_at"": ""2025-11-09T10:53:28.8...",12,2025-11-09 10:53:28.890822,2025-11-09 10:53:29.929344,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
13,True,200,"{""test"": 13, ""send_at"": ""2025-11-09T10:53:28.8...",13,2025-11-09 10:53:28.890899,2025-11-09 10:53:29.926742,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
14,True,200,"{""test"": 14, ""send_at"": ""2025-11-09T10:53:28.8...",14,2025-11-09 10:53:28.890959,2025-11-09 10:53:29.929905,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
15,True,200,"{""test"": 15, ""send_at"": ""2025-11-09T10:53:28.8...",15,2025-11-09 10:53:28.891017,2025-11-09 10:53:29.926860,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
16,True,200,"{""test"": 16, ""send_at"": ""2025-11-09T10:53:28.8...",16,2025-11-09 10:53:28.891148,2025-11-09 10:53:29.929725,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
17,True,200,"{""test"": 17, ""send_at"": ""2025-11-09T10:53:28.8...",17,2025-11-09 10:53:28.891220,2025-11-09 10:53:29.927783,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
18,True,200,"{""test"": 18, ""send_at"": ""2025-11-09T10:53:28.8...",18,2025-11-09 10:53:28.891279,2025-11-09 10:53:29.926996,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900
19,True,200,"{""test"": 19, ""send_at"": ""2025-11-09T10:53:28.8...",19,2025-11-09 10:53:28.891338,2025-11-09 10:53:29.927674,2025-11-09 10:53:28.900,2025-11-09 10:53:29.900


In [11]:
import pandas as pd
import json

# extract both fields from the JSON string
df["send_at"] = df["text"].apply(lambda t: json.loads(t)["send_at"])
df["end_time"] = df["text"].apply(lambda t: json.loads(t)["end_time"])

# convert to datetime
df["send_at"] = pd.to_datetime(df["send_at"])
df["end_time"] = pd.to_datetime(df["end_time"])

# round to 0.1s (one decimal of a second)
df["send_at_0_1s"] = df["send_at"].dt.round("100ms")
df["end_time_0_1s"] = df["end_time"].dt.round("100ms")

# assert both columns are consistent
assert df["send_at_0_1s"].nunique() == 1, f"send_at mismatch: {df['send_at_0_1s'].unique()}"
assert df["end_time_0_1s"].nunique() == 1, f"end_time mismatch: {df['end_time_0_1s'].unique()}"
assert df["ok"].all(), f"Some requests failed: {df[~df['ok']]}"
# new check for ok
failed_mask = ~df["ok"]
failed_count = failed_mask.sum()

assert failed_count == 0, f"{failed_count} requests are not True, indexes: {df.index[failed_mask].tolist()}"

print("✅ All checks passed: send_at and end_time align to 0.1s, and all responses are OK.")

✅ All checks passed: send_at and end_time align to 0.1s, and all responses are OK.


In [ ]:
# display(df)